# Prevue 
With Prevue, you are able to look up a specific show, movie, channel, subscription service, or person and get more information about them. 
* For shows, you will receive the genre, synopsis, channel, air date and time, and IMDb ID.
* For movies, you will receive the 10 most popular movies, their rating, release date, if it is still in theaters, and IMDb ID.
* For channel, you will be able to view the 10 most popular shows on that channel, when they first aired, and their IMDb ID.
* For subscription service, you will be able 10 most popular shows or movies provided by the serivce.

Happy searching! -
Bryan Jardines & Gizelle Jayne Vidal

In [8]:
# Please run the next two cells before starting.
!pip install guidebox

import csv
import guidebox
import json
import pandas as pd
import requests
from IPython.display import display,HTML

In [6]:
guidebox.api_key = 'b6b51fbc578321cbd4966880cbce6ea9537a9ce4'

In [7]:
# Run this cell to use the program.
display(HTML("<span style='color:purple;font-size: 30px; font-weight:bold;'"'Welcome to Prevue!'))

# importing csv file for the list of two-letter country codes
dic = {}
file_name = 'wikipedia-iso-country-codes.csv'
with open(file_name) as f: # opening the file
    file = csv.DictReader(f, delimiter=',')
    for line in file:
        dic[line['English short name lower case']] = line['Alpha-2 code']

# setting the region
while True:
    print('First, enter the country where you are currently streaming or watching television.')
    country = input('What country are you in? ')
    try:
        dic[country] # if the user entered a valid region, then there should be a corresponding key-value pair
        guidebox.region = "%s" % (dic[country]) # setting the matching value - the two-letter country code - as the region
        print('The region is set as: %s. \n' % (guidebox.region))
        break
    except KeyError:
        print('That is not a valid region. Please try again. \n')

# choosing a branch
while True:
    branch = input('What are you looking for (show, movie, channel, or streaming service)? To quit the program, enter "quit". ')
    print('You have chosen: %s. \n' % (branch))
    
    # branch - quit
    if branch == 'quit':
        print('Thank you for using Prevue.')
        break

    # branch - show
    elif branch == 'show':
        show_query = input('Enter the name of the show you would like to know more about: ')
        try:
            show_search = guidebox.Search.shows(field='title', query=show_query)
            # finding the id from the first result within the json object and assigning it to a variable
            show_id = show_search['results'][0]['id']
            show_results = guidebox.Show.retrieve(id=show_id)
            show_data = {'Title': show_results['title'], 'Genre': show_results['genres'][0]['title'], 'Rating' : show_results['rating'], 'Channel': show_results['channels'][0]['name'], 'Air Date': show_results['air_day_of_week'], 'Air Time': show_results['air_time']}
            index = [0]
            show_df = pd.DataFrame(show_data, index)
            print(show_df)
            print('\nSynopsis: %s' % (show_results['overview']))
        except:
            print('%s is not a valid entry. Please try again.\n' % (show_query))
            
    # branch - movie
    elif branch == 'movie':
        movies = guidebox.Movie.list(limit=10)  
        # print(movies)
        try:
            # initilazing variables
            movie_data = {}
            movie_titles = []
            movie_ratings = []
            movie_release_dates = []
            movie_theaters = []
            movie_id = []
            i = 0
            # print(movie_data)
            #for loop to read through the json object and assign words to lists
            for i in range(0,10):
                movie_titles.append(movies['results'][i]['original_title'])
                movie_ratings.append(movies['results'][i]['rating'])
                movie_release_dates.append(movies['results'][i]['release_date'])
                movie_theaters.append(movies['results'][i]['in_theaters'])
                movie_id.append(movies['results'][i]['imdb'])
                i += 1
            #creating key-value pairs
            movie_data = {'Titiles': movie_titles, 'Rating': movie_ratings, 'Release Date': movie_release_dates, 'In Theaters?': movie_theaters, 'IMDb ID': movie_id}
            #creating a dataframe and printing the dataframe
            movie_df = pd.DataFrame(movie_data)
            print(movie_df)
        except:
            print('Error occured. Please try again.\n')

    # branch - channel
    elif branch == 'channel':
        channel = input('Enter the name of the channel you would like to view: ')
        try:
            channel_shows = guidebox.Show.list(channel=channel, limit=10)
            channel_dic = {}
            channel_titles = []
            channel_date = []
            channel_id = []
            i = 0
            #for loop to read through the json object and assign words to lists
            for i in range(0,10):
                channel_titles.append(channel_shows['results'][i]['title'])
                channel_date.append(channel_shows['results'][i]['first_aired'])
                channel_id.append(channel_shows['results'][i]['imdb_id'])
                i += 1
            #creating key-value pairs
            channel_dic = {'Titles': channel_titles, 'First Air Date': channel_date, 'IMDb ID': channel_id}
            #creating a dataframe and printing the dataframe
            channel_df = pd.DataFrame(channel_dic)
            print(channel_df)
            print('')
            channel_dic.clear()
        except:
            print('%s is not a valid entry. Please try again.' % (channel))
            
    # branch - streaming service
    elif branch == 'streaming service':
        streaming = input('Enter the name of your streaming service: ').lower()
        # normalizing the text so that it can be used in the API
        streaming = streaming.split(' ')
        source = '_'.join(streaming)
        try:
            stream_shows = guidebox.Show.list(sources=source, limit=10)
            stream_dic = {}
            stream_titles = []
            stream_date = []
            stream_id = []
            i = 0
            # for loop to read through the json object and assign words to lists
            for i in range(0,10):
                stream_titles.append(stream_shows['results'][i]['title'])
                stream_date.append(stream_shows['results'][i]['first_aired'])
                stream_id.append(stream_shows['results'][i]['imdb_id'])
                i += 1
            # creating key-value pairs
            stream_dic = {'Titles': stream_titles, 'First Air Date': stream_date, 'IMDb ID': stream_id}
            # creating a dataframe and printing the dataframe
            stream_df = pd.DataFrame(stream_dic)
            print(stream_df)
            print('')
            stream_dic.clear()
        except:
            print('%s is not a valid entry. Please try again.' % (streaming))

    # branch - person
 #   elif branch == 'person':
 #       name = input('What is the name of the celebrity, actor/actress, or director you are looking for? ')
 #       try:
 #           person = guidebox.Search.person(query=name)
 #           person_id = person['results'][0]['id']
 #           credits_crew = guidebox.Person.credits(id=person_id, role='crew')
 #           credits_cast = guidebox.Person.credits(id=person_id, role='cast')
 #           print(credits_crew)
 #           print(credits_cast)
 #       except:
 #           print('An unknown error occured. Please try again.')
        
    # branch - user does not quit or enter a branch name
    else:
        print('That is not a valid entry. Please try again.')

SyntaxError: invalid syntax (<ipython-input-7-bc1269e43ef2>, line 4)